In [1]:
import sys
sys.path.append("D:/ML_Study/2017-CCF-BDCI-AIJudge")

In [2]:
# -*- encoding:utf-8 -*-
import codecs
import subprocess
from collections import namedtuple

import numpy as np
import pandas as pd
from gensim.models import Doc2Vec
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from utils import LOGGER
from config.db_config import Config
import warnings 
warnings.filterwarnings('ignore')
config = Config()

D:\opt\anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\opt\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 加载数据

In [3]:
data_df = pd.read_csv(config.data_csv_path, encoding='utf-8')
y = data_df['penalty'] - 1
rows = data_df.shape[0]

## 准备doc2vec 格式数据

In [4]:
SentimentDocument = namedtuple('SentimentDocument', 'words tags')
class Doc_list(object):

    def __init__(self, f):
        self.f = f
    def __iter__(self):
        for i,line in enumerate(codecs.open(self.f,encoding='utf8')):
            words = line.strip().split(' ')
            tags = [int(words[0][2:])]
            words = words[1:]
            yield SentimentDocument(words,tags) 
            
def write_d2v_data_path(content_list):
    doc_f = codecs.open(config.feat_d2v, encoding='utf-8', mode='w')
    for i, content in enumerate(content_list):
        if i % 10000 == 0:
            LOGGER.log('iter=%d' % i)
        doc_f.write(u'_*{} {}\n'.format(i, content))
    doc_f.close()

write_d2v_data_path(data_df['content'])

2019-04-01 15:59:49 iter=0


In [5]:
text="_*9 公诉 机关 平湖市 人民检察院 被告人 童某 个体 拆房 因涉嫌 盗窃罪 2012 平湖市 公安局 刑事拘留 同年 依法逮捕 羁押于 本市 看守所 平湖市 人民检察院 以平检 刑诉 2012 525 起诉书 指控 被告人 某犯 盗窃罪 2012 日向 本院 提起公诉 本院 同日 受理 依法 简易程序 实行 独任 审判 公开 开庭审理 本案 被告人 童某 到庭 参加 诉讼 现已 审理 终结 2008 时许 被告人 童某 童得明 判刑 本市 曹桥 街道 愚桥 村大 竹园 废品收购 被告人 童某 路边 望风 童得明 采用 搭线 手段 得失 停放 该处 华益牌 电瓶 三轮车 一辆 价值 2475 查明 赃物 电瓶 三轮车 扣押 发还 失主 上述事实 被告人 童某 开庭审理 过程 无异议 接受 刑事案件 登记表 失主 陈述 证人 证言 同伙 供述 刑事 判决书 辨认 笔录 照片 身份证明 搜查 笔录 扣押 发还 物品 清单 价格 鉴定结论 抓获 证据 予以 证实 足以认定"

In [6]:
words = text.strip().split(' ')
tags = [int(words[0][2:])]
words = words[1:]
print(words[0:4])
print(tags)
sd = SentimentDocument(words,tags)
sd

['公诉', '机关', '平湖市', '人民检察院']
[9]


SentimentDocument(words=['公诉', '机关', '平湖市', '人民检察院', '被告人', '童某', '个体', '拆房', '因涉嫌', '盗窃罪', '2012', '平湖市', '公安局', '刑事拘留', '同年', '依法逮捕', '羁押于', '本市', '看守所', '平湖市', '人民检察院', '以平检', '刑诉', '2012', '525', '起诉书', '指控', '被告人', '某犯', '盗窃罪', '2012', '日向', '本院', '提起公诉', '本院', '同日', '受理', '依法', '简易程序', '实行', '独任', '审判', '公开', '开庭审理', '本案', '被告人', '童某', '到庭', '参加', '诉讼', '现已', '审理', '终结', '2008', '时许', '被告人', '童某', '童得明', '判刑', '本市', '曹桥', '街道', '愚桥', '村大', '竹园', '废品收购', '被告人', '童某', '路边', '望风', '童得明', '采用', '搭线', '手段', '得失', '停放', '该处', '华益牌', '电瓶', '三轮车', '一辆', '价值', '2475', '查明', '赃物', '电瓶', '三轮车', '扣押', '发还', '失主', '上述事实', '被告人', '童某', '开庭审理', '过程', '无异议', '接受', '刑事案件', '登记表', '失主', '陈述', '证人', '证言', '同伙', '供述', '刑事', '判决书', '辨认', '笔录', '照片', '身份证明', '搜查', '笔录', '扣押', '发还', '物品', '清单', '价格', '鉴定结论', '抓获', '证据', '予以', '证实', '足以认定'], tags=[9])

In [7]:
doc_list = Doc_list(config.feat_d2v)

In [9]:
doc_list

In [11]:
config.feat_d2v

'D:/ML_Study/2017-CCF-BDCI-AIJudge/data/output/corpus/doc_for_d2v_12w.txt'

## distributed bag of words －doc2vec 模型训练和预测

In [14]:
print("distributed bag of words ..........")
d2v = Doc2Vec(dm=0, size=300, negative=5, hs=0, min_count=3, window=30, sample=1e-5, workers=8, alpha=0.025,min_alpha=0.025)
d2v.build_vocab(doc_list)

for i in range(5):
    LOGGER.log('pass:' + str(i))
    d2v.train(doc_list, total_examples=d2v.corpus_count, epochs=d2v.iter)
    X_d2v = np.array([d2v.docvecs[i] for i in range(rows)])
    '''
        Evaluate a score by cross-validation
        Array of scores of the estimator for each run of the cross validation

        scores：返回cv＝6 每次的得分
    '''
    scores = cross_val_score(LogisticRegression(C=3.0),X_d2v,y,cv=5) #ross_val_score(lasso, X, y)
    LOGGER.log('dbow: ' + str(scores) + ' ' + str(np.mean(scores)))

distributed bag of words ..........
2019-04-01 16:04:27 pass:0
2019-04-01 16:04:28 dbow: [0.34782609 0.31818182 0.19047619 0.33333333 0.5       ] 0.3379634857895727
2019-04-01 16:04:28 pass:1
2019-04-01 16:04:28 dbow: [0.43478261 0.31818182 0.23809524 0.33333333 0.5       ] 0.36487859966120834
2019-04-01 16:04:28 pass:2
2019-04-01 16:04:28 dbow: [0.43478261 0.31818182 0.23809524 0.33333333 0.5       ] 0.36487859966120834
2019-04-01 16:04:28 pass:3
2019-04-01 16:04:28 dbow: [0.30434783 0.40909091 0.28571429 0.33333333 0.5       ] 0.366497270845097
2019-04-01 16:04:28 pass:4
2019-04-01 16:04:29 dbow: [0.34782609 0.31818182 0.19047619 0.33333333 0.5       ] 0.3379634857895727


In [15]:
d2v.save(config.model_d2v_dbow)
LOGGER.log('Save done!')

2019-04-01 16:04:33 Save done!


In [16]:
# 获取编号“0” 文档的向量
d2v.docvecs[0]

array([-8.61452520e-03,  3.63759287e-02,  1.66141599e-01,  1.52416378e-01,
        1.98822655e-02,  3.54253352e-02,  7.25035295e-02, -1.91939380e-02,
       -1.23215444e-01,  1.55254349e-01, -3.63544375e-02, -4.37453352e-02,
        1.54316768e-01, -2.35546902e-02,  1.77426249e-04, -2.44019125e-02,
       -1.73810691e-01, -2.46189442e-02,  7.23381415e-02, -8.76142383e-02,
        2.90353626e-01,  1.00330219e-01, -1.15160242e-01,  3.09184995e-02,
        1.65367544e-01, -5.62796555e-02,  1.26652360e-01,  9.68257859e-02,
        1.33450732e-01, -1.42237067e-01,  3.53322066e-02, -1.42342970e-01,
       -1.27665281e-01, -2.64664739e-01,  5.25391623e-02,  6.04883581e-02,
        5.88680282e-02, -2.25655526e-01,  1.01206422e-01,  1.67621560e-02,
       -1.73885059e-02,  1.55603826e-01, -1.75726965e-01, -1.07444257e-01,
        1.24117270e-01, -4.07234244e-02,  1.45949617e-01,  1.45224899e-01,
        4.00274200e-03,  1.46330714e-01, -1.66181013e-01, -4.07856517e-02,
       -4.71424162e-02,  

## distributed memory -doc2vec 模型训练和预测

In [17]:
d2v = Doc2Vec(dm=1, size=300, negative=5, hs=0, min_count=3, window=30, sample=1e-5, workers=8, alpha=0.025,
              min_alpha=0.025)
d2v.build_vocab(doc_list)
for i in range(5):
    LOGGER.log('pass:' + str(i))
    d2v.train(doc_list, total_examples=d2v.corpus_count, epochs=d2v.iter)
    X_d2v = np.array([d2v.docvecs[i] for i in range(rows)])
    scores = cross_val_score(LogisticRegression(C=3.0), X_d2v, y, cv=5)  # ross_val_score(lasso, X, y)
    LOGGER.log('dm: ' + str(scores) + ' ' + str(np.mean(scores)))

2019-04-01 16:04:34 pass:0
2019-04-01 16:04:35 dm: [0.34782609 0.27272727 0.23809524 0.38888889 0.5625    ] 0.3620074973335843
2019-04-01 16:04:35 pass:1
2019-04-01 16:04:35 dm: [0.30434783 0.31818182 0.19047619 0.33333333 0.5       ] 0.3292678336156597
2019-04-01 16:04:35 pass:2
2019-04-01 16:04:35 dm: [0.34782609 0.31818182 0.23809524 0.33333333 0.5       ] 0.34748729531338224
2019-04-01 16:04:35 pass:3
2019-04-01 16:04:35 dm: [0.34782609 0.31818182 0.23809524 0.33333333 0.5       ] 0.34748729531338224
2019-04-01 16:04:35 pass:4
2019-04-01 16:04:36 dm: [0.34782609 0.31818182 0.23809524 0.33333333 0.5       ] 0.34748729531338224


In [18]:
d2v.save(config.model_d2v_dm)
LOGGER.log('Save done!')

2019-04-01 16:04:42 Save done!
